In [ ]:
region = 'charlie'
organization_id = '96902'
tenant_ids = None
ioc_file = 'ioc_threat_hunt'
days = 3
TAEGIS_MAGIC_NOTEBOOK_FILENAME = None

In [ ]:
%load_ext taegis_magic

from taegis_magic.core.log import get_module_logger, TRACE_LOG_LEVEL
logger = get_module_logger()
logger.setLevel(TRACE_LOG_LEVEL)
    
from pathlib import Path

from taegis_magic.core.service import get_service
from taegis_magic.pandas.tenants import lookup_first_environment
from taegis_magic.pandas.ioc import threaded_multi_tenant_ioc_search

In [ ]:
service = get_service(tenant_id=organization_id, environment=region)

In [ ]:
%taegis auth login --use-universal-authentication

In [ ]:
tenants_search_filters = []
if organization_id:
    tenants_search_filters.append(f"--filter-by-tenant-hierarchy {organization_id}")
if tenant_ids:
    for tenant_id in tenant_ids:
        tenants_search_filters.append(f"--filter-by-tenant {tenant_id}")
tenants_search_filter = ' '.join(tenants_search_filters)
tenants_search_filter

In [ ]:
%taegis tenants search --tenant "$organization_id" \
    $tenants_search_filter \
    --region "$region" \
    --assign tenants

In [ ]:
tenants['first_environment'] = tenants.pipe(lookup_first_environment)
tenants[['id', 'name', 'first_environment']]

In [ ]:
for id_, environment in tenants[['id', 'first_environment']].itertuples(index=False):
    print(f"Tenant ID: {id_}, Environment: {environment}")

In [ ]:
iocs_file = Path(ioc_file)
if not iocs_file.exists():
    raise FileNotFoundError(f"IOCs file '{iocs_file}' does not exist.")

iocs = iocs_file.read_text().splitlines()
iocs

In [ ]:
results = threaded_multi_tenant_ioc_search(
    service=service,
    tenants=tenants,
    iocs=iocs,
    days=days,
)
results

In [ ]:
results.to_parquet(f"{Path(TAEGIS_MAGIC_NOTEBOOK_FILENAME).stem}.parquet")